# Import libraries 



In [ ]:
import pandas as pd
from tqdm import tqdm
from tqdm.notebook import tqdm_notebook
import numpy as np
import pandas as pd
import numpy as np
import re
import os


# Read the Csv file 

In [ ]:
df = pd.read_csv('/content/unit_cell_catalog2000.csv')

Save the properties(a,b,c,alpha,beta,gamma) in prop_df and we will get back to that in the training

In [147]:
prop_df = df.iloc[: , 2:8]
prop_df

,a,b,c,alpha,beta,gamma
0,1.00000,1.00000,1.00000,90,90.0,90
1,0.85075,0.57449,1.00000,90,90.0,90
2,1.00000,1.00000,1.00000,90,90.0,90
3,0.99471,0.99471,1.00000,90,90.0,120
4,0.50300,0.50300,1.00000,90,90.0,90
...,...,...,...,...,...,...
1995,0.68452,1.00000,0.34059,90,90.0,90
1996,0.99994,1.00000,0.80392,90,90.2,90
1997,0.38120,0.44449,1.00000,90,90.0,90
1998,0.22690,0.22690,1.00000,90,90.0,90


Same here to the output : Save the output(cx,cy,cz,nx,ny,nz) in y  

In [ ]:
y = df.iloc[: , 21:27]
y

Extracting coordinates for adjacency matrix

Classify into first_nodes for first group and sec_node for the second
split the x, y and z nodes

In [ ]:
from numpy.core.fromnumeric import shape

column_names = df.columns.values.tolist()
column_dict = {}
for i in column_names:
    column_dict[column_names.index(i)] = i

nodal_coords = df.iloc[: , column_names.index("Nodal_nrow"):column_names.index("Nodal_ncol")+1]

bar_coords = df.iloc[: , column_names.index("Bar_nrow"):column_names.index("Bar_ncol")+1]

nodal_positions = pd.DataFrame()
bar_connectivity = pd.DataFrame()
first_nodes = pd.DataFrame()
second_nodes = pd.DataFrame()
x_nodes = pd.DataFrame()
y_nodes = pd.DataFrame()
z_nodes = pd.DataFrame()
for index, column_name in column_dict.items():
    if re.search("^nodal_positions", column_name):
        nodal_positions[column_name] = df.iloc[: , index]
        node_num = int(re.search("^nodal_positions_(\d+?)_", column_name).group(1))
        if column_name.endswith("1"):
            
            x_nodes[node_num] = df.iloc[: , index]

        if column_name.endswith("2"):
            y_nodes[node_num] = df.iloc[: , index]

        if column_name.endswith("3"):
            z_nodes[node_num] = df.iloc[: , index]

    if re.search("^bar_connectivity", column_name):
        bar_connectivity[column_name] = df.iloc[: , index]
        bar_num = int(re.search("^bar_connectivity_(\d+?)_", column_name).group(1))
        if column_name.endswith("1"):
            first_nodes[bar_num] = df.iloc[: , index]
        if column_name.endswith("2"):
            second_nodes[bar_num] = df.iloc[: , index]

x_nodes = x_nodes.fillna(0)
y_nodes = y_nodes.fillna(0)
z_nodes = z_nodes.fillna(0)
first_nodes = first_nodes.fillna(0)
second_nodes = second_nodes.fillna(0)

# Now we want to calculate the distace between atoms (nodes) 

In [ ]:
import math

distances = pd.DataFrame()
i_df = pd.DataFrame()
m_df = pd.DataFrame()
n_df = pd.DataFrame()
for (index1, s1), (index2, s2) in zip(first_nodes.iterrows(), second_nodes.iterrows()):
    for node_number in list(first_nodes.columns):
        node1 = int(s1[node_number])
        node2 = int(s2[node_number])

        try:
            x1 = x_nodes[node1][index1]
            x2 = x_nodes[node2][index2]

            y1 = y_nodes[node1][index1]
            y2 = y_nodes[node2][index2]

            z1 = z_nodes[node1][index1]
            z2 = z_nodes[node2][index2]
            
            edge_length = math.sqrt((x2 - x1)**2 + (y2 - y1)**2 + (z2 - z1)**2)

            i = (x2-x1) / edge_length
            m = (y2 - y1) / edge_length
            n = (z2 - z1) / edge_length
        except:
            edge_length = 0

            i, m, n = 0, 0, 0

        distances[node_number] = edge_length
        i_df[node_number] = edge_length
        m_df[node_number] = edge_length
        n_df[node_number] = edge_length

Build a dataframe to contain all computed data

In [ ]:
model_data = pd.concat([prop_df, distances, i_df, m_df, n_df], axis=1).to_dict()

## Modeling

Install All necessary modules

In [ ]:
#!pip install pymatgen
#!pip install mendeleev 
#!pip install graphviz 
#!pip install pydot 
#!pip install sklearn
#!pip install crysnet

In [ ]:
from crysnet.data import Dataset
from crysnet.data.generator import GraphGenerator
from crysnet.models import GNN
from crysnet.models.gnnmodel import CgcnnModel
from pymatgen.core.structure import Structure
from pathlib import Path

In [ ]:
ModulePath = Path('__file__').parent.absolute() # workdir

In [ ]:
dataset = Dataset(task_type='multiclassification', data_path=ModulePath)
BATCH_SIZE = 1600
DATA_SIZE = None
CUTOFF = 2.5

#structure the dataset for consumption by model
dataset.prepare_x(model_data)
dataset.prepare_y()

In [ ]:
Generators = GraphGenerator(dataset, data_size=DATA_SIZE, batch_size=BATCH_SIZE, cutoff=CUTOFF)
train_data = Generators.train_generator
valid_data = Generators.valid_generator
test_data = Generators.test_generator
multiclassification = Generators.multiclassification  

In [ ]:
gnn = GNN(model=CgcnnModel, atom_dim=16, bond_dim=64, num_atom=118, 
          state_dim=16, sp_dim=230, units=32, edge_steps=1, 
          message_steps=1, transform_steps=1, num_attention_heads=8, 
          dense_units=64, output_dim=64, readout_units=64, dropout=0.0, 
          reg0=0.00, reg1=0.00, reg2=0.00, reg3=0.00, reg_rec=0.00, batch_size=BATCH_SIZE, 
          spherical_harmonics=True, regression=dataset.regression, optimizer = 'Adam')

In [ ]:
try:
  gnn.train(train_data, valid_data, test_data, epochs=700, lr=3e-3, warm_up=True, load_weights=False, verbose=1, checkpoints=None, save_weights_only=True, workdir=ModulePath)
  gnn.predict_datas(test_data, workdir=ModulePath)    # predict on test datas with labels
except Exception as err:
  print(err)
y_pred_keras = gnn.predict(datas)